# Dataset Iris

### Exploração simples

O **Dataset Iris** é bastante comum de ser utilizado para explicar alguns recursos de *Machine Learning*. Aqui não será diferente, e utilizaremos para dar base à nossa atividade de hoje.

No primeiro momento, vamos analisar rapidamente este conjunto de dados, e ver duas de suas variáveis, a **Petal.Length** e a **Petal.Width** com a geração de um gráfico de dispersão.

Após a geração do gráfico, vamos recriar o dataset em um novo objeto. O objetivo é observar somente os dados e com isso, aplicar um modelo de Aprendizado Não Supervisionado. É esperado que o algoritmo encontre valores similares aos reais. Para finalizar, vamos comparar o resultado do modelo criado com o que já possuímos da exploração inicial.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dados = pd.read_csv("Dataset/iris.csv", header=None, names=['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width','Species'])

In [ ]:
dados.info()

In [ ]:
dados.describe()

In [ ]:
dados.head()

In [ ]:
cores = {'setosa':'tab:cyan','versicolor':'tab:purple','virginica':'tab:green'}

for especies in dados['Species'].unique():
    tmp = dados[dados['Species'] == especies]
    plt.scatter(tmp['Petal.Length'], tmp['Petal.Width'], color=cores[especies] )

### K-Means com PySpark

In [ ]:
# !pip install pyspark

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark import SparkContext, SparkConf

from numpy import array
from math import sqrt


In [ ]:
conf = SparkConf().setMaster('local').setAppName('PySpark Intro')
sc = SparkContext.getOrCreate(conf = conf)

In [ ]:
sc

In [ ]:
dadosSpark = sc.textFile("iris.csv")

In [ ]:
dadosSpark.take(5)

In [ ]:
dadosSpark = dadosSpark.map(lambda line: array([x for x in line.split(',')]) )

In [ ]:
dadosSpark

In [ ]:
dadosSpark.take(5)

In [ ]:
#'Petal.Length' e 'Petal.Width'
dadosSemSpecie = dadosSpark.map(lambda x : array([ float(x[2]),  float(x[3]) ]))

In [ ]:
dadosSemSpecie

In [ ]:
dadosSemSpecie.take(5)

In [ ]:
KMeansCluster = KMeans.train(rdd=dadosSemSpecie,k=3,maxIterations=100,initializationMode='random')

In [ ]:
KMeansCluster.centers

In [ ]:
resultado = KMeansCluster.predict(dadosSemSpecie)
resultado.take(5)

In [ ]:
grupos = resultado.distinct()
grupos.take(3)

In [ ]:
retorno = resultado.map(lambda x: (x)).collect()

In [ ]:
arRetorno = array(retorno)
dados['cluster'] = arRetorno

In [ ]:
cores = {0:'tab:cyan',1:'tab:purple',2:'tab:green'}

for grupo in dados['cluster']:
    tmp = dados[dados['cluster'] == grupo]
    plt.scatter(tmp['Petal.Length'], tmp['Petal.Width'], color=cores[grupo] )


In [ ]:
cores = {'setosa':'tab:cyan','versicolor':'tab:purple','virginica':'tab:green'}

for especies in dados['Species'].unique():
    tmp = dados[dados['Species'] == especies]
    plt.scatter(tmp['Petal.Length'], tmp['Petal.Width'], color=cores[especies] )

In [ ]:
cores = {0:'tab:cyan',1:'tab:purple',2:'tab:green'}

for grupo in dados['cluster']:
    tmp = dados[dados['cluster'] == grupo]
    plt.scatter(tmp['Petal.Length'], tmp['Petal.Width'], color=cores[grupo] )
    
centros = array(KMeansCluster.centers)
centros = pd.DataFrame(centros)

plt.scatter(centros[0],centros[1], marker="^", color="black")
